In [11]:
import os
import json
import numpy as np
import pandas as pd
import re
import statistics

# with open('config.json') as infile:
#     config = json.load(infile)
    
# repo = config['desktop']

repo = 'C:/Users/possi/OneDrive - University of Glasgow/data/FMAS2024_RAW_rev'

# Model checking plan data

In [22]:
def extract_plan_data(run):
    data = []
    file = open(repo+'/{}/autoctrl.txt'.format(run), 'r')
    
    for line in file:
        if line.split(':')[0] == 'ACCEPT':
            data.append(line.split(':')[-1].strip())
        elif line.split(':')[0] == 'PATH':
            data.append(line.split(':')[-1].strip())
        elif line.split(':')[0] == 'PLAN':
            data.append(line.split(':')[-1].strip())
        elif line.strip()[-2:] == 'ms':
            d = line.split('!')[1].strip()[:-2]
            data.append(float(d))
            
    file.close()
    
    if len(data) == 0:
        data = [None] * 8
    
    if len(data) == 4:
        data += [None] * 4

    return data
            
def add_plan_to_run_data(n_runs):
    count = 0
    for run in os.listdir(repo):    
        # with open(repo+'/{}/{}.json'.format(run, run), 'r') as infile:
        #     run_data = json.load(infile)

        log_data = extract_plan_data('mc3')
        print(log_data)
        return
    #     run_data['accept1'] = log_data[0]
    #     run_data['path1'] = log_data[1]
    #     run_data['plan1'] = log_data[2]
    #     if (log_data[2] != None):
    #         run_data['steps1'] = int(len(log_data[2].split('->')[:-1]))
    #     else:
    #         run_data['steps1'] = log_data[2]
    #     run_data['latency1'] = log_data[3]
    #     run_data['accept2'] = log_data[4]
    #     run_data['path2'] = log_data[5]
    #     run_data['plan2'] = log_data[6]
    #     if (log_data[6] != None):
    #         run_data['steps2'] = int(len(log_data[6].split('->')[:-1]))
    #     else:
    #         run_data['steps2'] = log_data[6]
    #     run_data['latency2'] = log_data[7]

    #     with open(repo+'/{}/{}.json'.format(run, run), 'w') as outfile: 
    #         json.dump(run_data, outfile)
            
    #     count += 1

    # print(run_data)
      
    try:
        assert count == n_runs
    except AssertionError:
        print('FAILED: count = {}'.format(count))
    
add_plan_to_run_data(n_runs=6)

['3', '0 1', 'left -> default', 7.293, '14', '0 2 13', 'right -> right -> default', 6.315, '14', '0 1 13', 'left -> left -> default', 9.899, '14', '0 1 13', 'left -> left -> default', 7.551, '14', '0 1 13', 'left -> left -> default', 7.155, '7 11', '0 1 3 5', 'left -> straight -> right -> default', 14.861, '14', '0 2 13', 'right -> right -> default', 5.39, '14', '0 2 13', 'right -> right -> default', 6.946, '14', '0 2 13', 'right -> right -> default', 7.662, '14', '0 1 13', 'left -> left -> default', 9.336, '14', '0 2 13', 'right -> right -> default', 5.975, '14', '0 2 13', 'right -> right -> default', 5.915, '8 12', '0 2 4 10', 'right -> straight -> right -> default', 5.456, '12', '0 2 4 10', 'right -> straight -> right -> default', 10.296, '8 12', '0 2 4 6', 'right -> straight -> left -> default', 7.352, '11', '0 1 3 9', 'left -> straight -> left -> default', 10.183, '4', '0 2', 'right -> default', 3.508, '14', '0 2 13', 'right -> right -> default', 8.005, '14', '0 1 13', 'left -> le

# Model checking resource usage

In [26]:
def _extract_stack_data(data):
    result = []
    
    for item in data:
        if item[:3] != 'SET':
            result.append(item.strip())
        else:
            break
            
    result = result[2:-2]
    # get stack size data here
    result = [int(x[-1]) for x in result]
            
    return result
    
def _extract_set_data(data):
    result = []
    
    for item in reversed(data[:-5]):
        if item[:3] != 'SET':
            try:
                result.append(int(item.strip()))
            except ValueError:
                pass
        else:
            break
        
    return result
    
def extract_performance_data(run):
    perf = []
    
    for file in os.listdir(repo+'\{}'.format(run)):
        if file[:3] == 'dfs':
            with open(repo+'\{}\{}'.format(run, file)) as infile:
                perf.append(infile.readlines())
         
    set_data = []
    stack_data = []
    
    for data in perf:
        stack_data.append(max(_extract_stack_data(data)))
        set_data.append(max(_extract_set_data(data)))
    
    return stack_data, set_data

def add_performance_to_run_data(n_runs):
    count = 0
    for run in os.listdir(repo):    
        with open(repo+'/{}/{}.json'.format(run, run), 'r') as infile:
            run_data = json.load(infile)
 
        stack_data, set_data = extract_performance_data(run)

        df = pd.DataFrame()
        df['stack_data'] = stack_data
        df['set_data'] = set_data
        df['state_size'] = 4
        df['n_states'] = 15
        df['adj_list_compile'] = 184
        df['stack_compile'] = 12
        df['set_compile'] = 24
        df.to_csv(repo+'/{}/{}model_check.csv'.format(run, run))
        
        # if len(stack_data) > 0:
        #     # max stack size and number of steps
        #     print(stack_data)
        #     run_data['max_stack_capacity'] = max(stack_data)
        #     run_data['min_stack_size'] = min(stack_data)
        #     run_data['mode_stack_size'] = statistics.mode(stack_data)
        # else:
        #     run_data['max_stack_capacity'] = None
        #     run_data['min_stack_size'] = None
        #     run_data['mode_stack_size'] = None
            
        # if len(set_data) > 0:
        #     run_data['max_set_size'] = max(set_data)
        #     run_data['min_set_size'] = min(set_data)
        #     run_data['mode_set_size'] = statistics.mode(set_data)
        # else:
        #     run_data['max_set_size'] = None
        #     run_data['min_set_size'] = None
        #     run_data['mode_set_size'] = None
            
        # with open(repo+'/{}/{}.json'.format(run, run), 'w') as outfile: 
        #     json.dump(run_data, outfile)
            
        count += 1
      
    try:
        assert count == n_runs
    except AssertionError:
        print('FAILED: count = {}'.format(count))

add_performance_to_run_data(n_runs=6)

# Process resource usage

In [16]:
def _extract_process_memory(run):
    swap = []
    physical = []
    process = []
    
    file = open(repo+'/{}/usage.txt'.format(run), 'r')
    
    for line in file:
        if line.split(':')[0] == 'MiB Mem ':
            physical.append(line.split(':')[-1].strip())
        elif line.split(':')[0] == 'MiB Swap':
            swap.append(line.split(':')[-1].strip())
        elif line.strip()[-8:] == 'autoctrl':
            process.append(line)

    file.close()

    # physical data
    physical_ = []
    physical = [x.split(',') for x in physical]
    for row in physical:
        physical_.append([float(x.strip()[:4]) for x in row])

    # swap data
    swap_ = []
    swap = [x.split(',') for x in swap]
    for row in swap:   
        swap_.append([x.strip() for x in row])

    swap__ = []
    for row in swap_:
        swap__.append([float(x.split()[0]) for x in row])

    # process data
    process_ = []
    for row in process:   
        process_.append(row.strip()[9:-8].strip().split(' '))

    process__ = []
    for row in process_:
        process__.append([x for x in row if x != ''])

    for i in range(len(process__)):    
        for j in range(len(process__[i])):
            if ':' in process__[i][j]:
                process__[i][j] = process__[i][j][-5:]
                
            try:
                process__[i][j] = float(process__[i][j])
            except ValueError:
                pass
                
    phy_cols = ['mem_total', 'mem_free', 'mem_used', 'cache']
    phy = pd.DataFrame(data=physical_, columns=phy_cols)

    swp_cols = ['swap_total', 'swap_free', 'swap_used']
    swp = pd.DataFrame(data=swap__, columns=swp_cols)

    proc_cols = ['pr', 'ni', 'virt', 'res', 'shr', 'status', 
                 'cpu_perc', 'mem_perc', 'cpu_time']
    proc = pd.DataFrame(data=process__, columns=proc_cols)

    df = pd.merge(proc, phy, how='inner', left_index=True, right_index=True)
    df = pd.merge(df, swp, how='inner', left_index=True, right_index=True)
    df['process_uptime'] = 0.001 * df.index
    df = df[df.mem_perc > 0]
    
    return df

def add_process_to_run_data(n_runs):
    count = 0
    for run in os.listdir(repo):    
        with open(repo+'/{}/{}.json'.format(run, run), 'r') as infile:
            run_data = json.load(infile)

        df = _extract_process_memory(run)
        df.to_csv(repo+'/{}/{}usage.csv'.format(run, run))

        count += 1
    try:
        assert count == n_runs
    except AssertionError:
        print('FAILED: count = {}'.format(count))

add_process_to_run_data(n_runs=6)